In [1]:
import torch
import torch.nn as nn
import numpy as np
from torchvision.models import mobilenet_v3_small, MobileNet_V3_Small_Weights
from torchvision.models import mobilenet_v3_large, MobileNet_V3_Large_Weights
import os

IMG_SIZE = 224
NUM_CLASSES = 3
CKPT_PATH = "models/mobilenetv3_large_best.pth"
ONNX_PATH = "models/mobilenetv3_large_best.onnx"
DEVICE = "cpu"


def build_model(num_classes=NUM_CLASSES, pretrained=True):
    weights = MobileNet_V3_Large_Weights.IMAGENET1K_V1 if pretrained else None
    model = mobilenet_v3_large(weights=weights)
    # weights = MobileNet_V3_Small_Weights.IMAGENET1K_V1 if pretrained else None
    # model = mobilenet_v3_small(weights=weights)
    if hasattr(model, 'classifier'):
        in_features = model.classifier[-1].in_features
        model.classifier[-1] = nn.Linear(in_features, num_classes)
    return model

model = build_model(num_classes=NUM_CLASSES, pretrained=False)
checkpoint = torch.load(CKPT_PATH, map_location=DEVICE)
model.load_state_dict(checkpoint["model_state"])
model.eval()


dummy_input = torch.randn(1, 3, IMG_SIZE, IMG_SIZE, device=DEVICE)

input_names = ["input"]
output_names = ["output"]

torch.onnx.export(
    model,
    dummy_input,
    ONNX_PATH,
    export_params=True,
    opset_version=11, 
    do_constant_folding=True,
    input_names=input_names,
    output_names=output_names,
    dynamic_axes={ "input": {0: "batch_size"}, "output": {0: "batch_size"} }
)

In [2]:
import onnx
onnx_model = onnx.load("model.onnx")
onnx.checker.check_model(onnx_model)

In [12]:
import tf2onnx
import onnx

onnx_model_path = "model.onnx"
saved_model_dir = "mobilenetv3_saved_model"

# Загружаем ONNX модель
onnx_model = onnx.load(onnx_model_path)

# Конвертация в TensorFlow SavedModel
tf2onnx.convert.from_onnx_model(
    onnx_model,
    output_path=saved_model_dir,
    output_format="saved_model",
    opset=11
)


AttributeError: module 'tf2onnx.convert' has no attribute 'from_onnx_model'